In [1]:
import sys
sys.path.append("../")

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#Plotting
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sns.set_context('paper')
import logging
logging.basicConfig(format='%(asctime)s %(levelname)-7s %(message)s',
                    stream=sys.stderr, level=logging.INFO)
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.INFO)

import os

from pathlib import Path
import pandas as pd
import json
import math

#In-house Module Imports
from config import Configuration 

In [4]:
def plot_all_analysis_line_plots(stats_df,param_dict,naming_dict, color_dict, use_std=True):
    
    strategies = stats_df['query_strategy'].unique()
    
    #fig = plt.figure(figsize=(param_dict['fig_width'], param_dict['fig_height']))
    fig, ax = plt.subplots(1,3,figsize=(param_dict['fig_width'], param_dict['fig_height']))

    for strategy in strategies:
        selected_score_data = stats_df.loc[stats_df['query_strategy'] == strategy]
        x = selected_score_data['query_iter']
        y = selected_score_data['mean']
        ax[0].plot(x, 
                 y,
                 linewidth=3,
                 color=color_dict[strategy],
                 label=strategy)
        if use_std:
            ax[0].fill_between(x, 
                             y - (0.5*selected_score_data['std']), 
                             y + (0.5*selected_score_data['std']),
                             alpha=0.2,  
                             color=color_dict[strategy],
                            )
        else:
            
            ax[0].fill_between(x, 
                 selected_score_data['ci95_lo'], 
                 selected_score_data['ci95_hi'],
                 alpha=1, 
                 color=color_dict[strategy],
                )

    legend = ax[0].legend(title="",prop={'size': param_dict['legend_size']})
    
    ax[0].title(f"{naming_dict['title_prefix']} w.r.t Number of Queries \n {naming_dict['fe_name']} - {naming_dict['system']} - {naming_dict['num_features']} \n #Known Apps in Training: {naming_dict['num_train_known_apps']}",fontsize=param_dict['title_size'])
    #plt.ylim([0.3, 1.01])
    ax[0].xlim([0, 250])

    ax[0].ylabel(f"{naming_dict['title_prefix']}",size=param_dict['y_label_font'])
    ax[0].xlabel("Number of Iterations",size=param_dict['x_label_font'])
    ax[0].xticks(fontsize=param_dict['x_ticks_font'])
    ax[0].yticks(fontsize=param_dict['y_ticks_font'])
    
    plt.show()

In [5]:
def plot_query_line_plot(stats_df,param_dict,naming_dict, color_dict, use_std=True):
    
    strategies = stats_df['query_strategy'].unique()

    fig = plt.figure(figsize=(param_dict['fig_width'], param_dict['fig_height']))

    for strategy in strategies:
        selected_score_data = stats_df.loc[stats_df['query_strategy'] == strategy]
        x = selected_score_data['query_iter']
        y = selected_score_data['mean']
        plt.plot(x, 
                 y,
                 linewidth=3,
                 color=color_dict[strategy],
                 label=strategy)
        if use_std:
            plt.fill_between(x, 
                             y - (0.5*selected_score_data['std']), 
                             y + (0.5*selected_score_data['std']),
                             alpha=0.2,  
                             color=color_dict[strategy],
                            )
        else:
            
            plt.fill_between(x, 
                 selected_score_data['ci95_lo'], 
                 selected_score_data['ci95_hi'],
                 alpha=1, 
                 color=color_dict[strategy],
                )

    legend = plt.legend(title="",prop={'size': param_dict['legend_size']})
    
    plt.title(f"{naming_dict['title_prefix']} w.r.t Number of Queries \n {naming_dict['fe_name']} - {naming_dict['system']} - {naming_dict['num_features']} \n #Known Apps in Training: {naming_dict['num_train_known_apps']}",fontsize=param_dict['title_size'])
    #plt.ylim([0.3, 1.01])
    plt.xlim([0, 250])

    plt.ylabel(f"{naming_dict['title_prefix']}",size=param_dict['y_label_font'])
    plt.xlabel("Number of Iterations",size=param_dict['x_label_font'])
    plt.xticks(fontsize=param_dict['x_ticks_font'])
    plt.yticks(fontsize=param_dict['y_ticks_font'])
    
    plt.show()

In [83]:
def calculate_stats(df, score_type):
    
    stats = df.groupby(['query_strategy','query_iter'])[score_type].agg(['mean', 'count', 'std'])

    ci95_hi = []
    ci95_lo = []
    z_star = 1.96 # 1.65: ci90

    for i in stats.index:
        m, c, s = stats.loc[i]#[2:5]
        ci95_hi.append(m + z_star*s/math.sqrt(c))
        ci95_lo.append(m - z_star*s/math.sqrt(c))

    stats['ci95_hi'] = ci95_hi
    stats['ci95_lo'] = ci95_lo

    stats.reset_index(['query_strategy','query_iter'], inplace=True)    
    
    return stats

In [8]:
#Update these
PARENT_DIR = 'active_learning_experiments'
SYSTEM = 'volta'
FE_NAME = 'tsfresh'
EXP_NAME = f'{FE_NAME}_experiments'
DIR_NAME_TO_GENERATE_RESULTS = 'exp_2_active_learning'

In [9]:
OUTPUT_DIR = f'/projectnb/peaclab-mon/{user}/{PARENT_DIR}' # or feature_extraction_experiments
CV_INDEX = 0
SCALER = 'None' #For now, do the scaling inside the notebook, then you can move that to the class function

In [ ]:
conf = Configuration(ipython=True,
                     overrides={
                         'output_dir': Path(OUTPUT_DIR), #change
                         'system' : SYSTEM,
                         'exp_name':EXP_NAME,                                                  
                         'cv_fold':CV_INDEX, 
                         'model_config': DIR_NAME_TO_GENERATE_RESULTS
                     })

In [ ]:
result_df = pd.DataFrame()
counter = 0
result_list = []

for cv_index in [0,1,2,3,4]:

    conf = Configuration(ipython=True,
                         overrides={
                             'output_dir': Path(OUTPUT_DIR), #change
                             'system' : SYSTEM,
                             'exp_name':EXP_NAME,                                                  
                             'cv_fold':cv_index, 
                             'model_config': DIR_NAME_TO_GENERATE_RESULTS
                         }
                        )

    for filename in os.listdir(conf['results_dir']):
        
        if  not "app-anom-selection" in filename:
            
            only_filename = filename.split('.')
            splitted_filename = only_filename[0].split('#')
            train_apps = splitted_filename[0].split(':')[1]
            test_apps = splitted_filename[1].split(':')[1]
            num_unknown_test_apps = len(test_apps.split("-"))
            num_known_train_apps = len(train_apps.split("-"))        

            temp_csv = pd.read_csv(Path(conf['results_dir']) / filename,index_col = [0])
            temp_csv['train_apps'] = train_apps
            temp_csv['test_apps'] = test_apps        
            temp_csv['num_unknown_test_apps'] = num_unknown_test_apps
            temp_csv['num_known_train_apps'] = num_known_train_apps
            temp_csv['repeat_num'] = int(splitted_filename[8])

            result_list.append(temp_csv)

            counter += 1
               
    result_df = pd.concat(result_list, axis=0)

In [ ]:
unique_query_strategies = result_df['query_strategy'].unique()
logging.info("Unique query strategies: %s",unique_query_strategies)
unique_methods = result_df['method'].unique()
logging.info("Unique Methods: %s", unique_methods)
unique_fe_methods = result_df['fe'].unique()
logging.info("Feature Extraction Methods: %s",unique_fe_methods)
unique_feature_counts = sorted(result_df['feature_count'].unique())
logging.info("Num Features: %s", unique_feature_counts)
unique_query_sizes = sorted(result_df['query_size'].unique())
logging.info("Unique query sizes: %s",unique_query_sizes)
unique_known_train_apps = result_df['num_known_train_apps'].unique()
logging.info("Number of Known Apps in the Training: %s", unique_known_train_apps)

# unique_random_selection = len(result_df[result_df['query_strategy'] == 'random']['repeat_num'].unique())
# logging.info("Number of Repeats for the Random Selection: %s", unique_random_selection)

unique_folds = len(result_df[result_df['query_strategy'] != 'random']['fold'].unique())
logging.info("Number of Folds for the Active Learning Methods: %s", unique_folds)

## Paper Plots

In [28]:
color_dict = {
                'entropy': 'tab:purple',
                'margin': 'tab:blue',    
                'uncertainty': 'orange',    
                'random': 'tab:green',
                'equal_app' : 'tab:red',
             }

param_dict = {
                'fig_width': 48,
                'fig_height': 12,
                'y_label_font': 40,
                'x_label_font': 40,
                'x_ticks_font': 37,
                'y_ticks_font': 37,    
                'legend_size': 42,
                'legend_title_size': 40,
                'title_size': 46,
                'title_pad': 40, 
                'fig_title_size': 50,
    
             }

In [29]:
selected_model = 'RandomForestClassifier'
num_query = 250
feature_count = 2000

In [30]:
naming_dict = {}
naming_dict['fe_name'] = FE_NAME
naming_dict['system'] = SYSTEM
naming_dict['num_features'] = int(feature_count)
naming_dict['num_queries'] = num_query  
naming_dict['model'] = selected_model

In [32]:
selected_result_df = result_df[(result_df['query_size'] == num_query) & (result_df['model'] == selected_model)]

In [ ]:
known_2_result_df = selected_result_df[selected_result_df['num_known_train_apps'] == 2]
known_4_result_df = selected_result_df[selected_result_df['num_known_train_apps'] == 4]
known_6_result_df = selected_result_df[selected_result_df['num_known_train_apps'] == 6]

# stats_far = calculate_stats(selected_result_df, 'false_alarm_rate')
# stats_amr = calculate_stats(selected_result_df, 'anomaly_miss_rate')

In [77]:
#sns.lineplot(data=mg_CoMD, x="macro_avg_f1_score", y="query_iter", hue="fold")

In [ ]:
#del result_df

In [84]:
known_2_stats_fscore = calculate_stats(known_2_result_df, 'macro_avg_f1_score')
known_4_stats_fscore = calculate_stats(known_4_result_df, 'macro_avg_f1_score')
known_6_stats_fscore = calculate_stats(known_6_result_df, 'macro_avg_f1_score')

In [85]:
def plot_multiple_query_line_plot(fscore_df, far_df, amr_df, param_dict, naming_dict, color_dict, num_query=50, use_std=False):
    
    
    fig, axs = plt.subplots(1,3,figsize=(param_dict['fig_width'], param_dict['fig_height']))
    
    for ind, (stats_df,fig_title, fig_y_label) in enumerate(zip([fscore_df,far_df,amr_df],["Training Dataset: 2 Apps","Training Dataset: 4 Apps","Training Dataset: 6 Apps"],["F1-score (Macro Avg)","",""])):
        
        strategies = stats_df['query_strategy'].unique()
        
        for strategy in strategies:
            
            selected_score_data = stats_df.loc[stats_df['query_strategy'] == strategy]
            x = selected_score_data['query_iter']
            y = selected_score_data['mean']
            axs[ind].plot(x, 
                     y,
                     linewidth=3,
                     color=color_dict[strategy],
                     label=strategy)
            if use_std:
                axs[ind].fill_between(x, 
                                 y - (1*selected_score_data['std']), 
                                 y + (1*selected_score_data['std']),
                                 alpha=0.2,  
                                 color=color_dict[strategy],
                                )
            else:

                axs[ind].fill_between(x, 
                     selected_score_data['ci95_lo'], 
                     selected_score_data['ci95_hi'],
                     alpha=0.1, 
                     color=color_dict[strategy],
                    )
                
        if ind == 0:
            axs[ind].axvline(x=50, linewidth=4, color='xkcd:black', linestyle='--', label="$Min Query_{F1_{95}}$")                      
        elif ind == 1:
            axs[ind].axvline(x=35, linewidth=4, color='xkcd:black', linestyle='--', label="$Min Query_{F1_{95}}$")                      
        elif ind == 2:
            axs[ind].axvline(x=30, linewidth=4, color='xkcd:black', linestyle='--', label="$Min Query_{F1_{95}}$")                      

            
        if naming_dict['system'] == 'eclipse':
            axs[ind].axhline(y=0.95, linewidth=4, color='r', linestyle='--', label='$F1_{95}$')                 
        elif naming_dict['system'] == 'volta':
            axs[ind].axhline(y=0.95, linewidth=4, color='r', linestyle='--', label='$F1_{95}$') 
            
            
        axs[ind].set_title(fig_title, fontsize=param_dict['title_size'])     
        axs[ind].set_xlim([0, num_query])
        
        axs[ind].set_ylabel(f"{fig_y_label}",size=param_dict['y_label_font'])
        axs[ind].set_xlabel("Number of Queries",size=param_dict['x_label_font'])
        axs[ind].tick_params(axis='x', labelsize=param_dict['x_ticks_font'])
        axs[ind].tick_params(axis='y', labelsize=param_dict['y_ticks_font'])                
    
    handles, labels = axs[0].get_legend_handles_labels()
    fig.legend(handles, ["Random","Uncertainty", "$Min Query_{F1_{95}}$", "$F1_{95}$"], loc='lower left', 
               bbox_to_anchor=(0.27, 1.0, 0.3, 0.4), ncol=5, frameon=True, mode='None',
               prop={'size': param_dict['legend_size']}
              )
        
    #fig.suptitle(f"Active Learning with {naming_dict['model']}",fontsize=param_dict['fig_title_size'])        
    fig.suptitle(f"Previously Unseen Applications - {naming_dict['system'].capitalize()}",fontsize=param_dict['fig_title_size'], y = 1.20) 
    plt.tight_layout()
    plt.savefig(f"/usr3/graduate/baksar/projectx/AI4HPCAnalytics/src/active_learning_experiments/plots/{naming_dict['system']}_{naming_dict['fe_name']}_unseen_apps.pdf", bbox_inches='tight')

In [ ]:
plot_multiple_query_line_plot(known_2_stats_fscore, known_4_stats_fscore, known_6_stats_fscore, 
                              param_dict, 
                              naming_dict, 
                              color_dict, 
                              num_query=num_query,
                              use_std=True)

## Plot for Each Feature Extraction Method and Rates Separately

In [27]:
num_train_known_apps = 8
feature_count = unique_feature_counts[0]
temp_result_df = result_df[result_df['num_known_train_apps'] == num_train_known_apps]

In [22]:
stats_fscore = calculate_stats(temp_result_df, 'macro_avg_f1_score')
stats_far = calculate_stats(temp_result_df, 'false_alarm_rate')
stats_amr = calculate_stats(temp_result_df, 'anomaly_miss_rate')

In [23]:
naming_dict = {}
naming_dict['fe_name'] = FE_NAME
naming_dict['system'] = SYSTEM
naming_dict['num_features'] = int(feature_count)
naming_dict['num_queries'] = unique_query_sizes[0]
naming_dict['num_train_known_apps'] = int(num_train_known_apps)

In [24]:
color_dict = {
                'entropy': 'tab:purple',
                'margin': 'tab:blue',    
                'uncertainty': 'orange',    
                'random': 'tab:green',
             }

In [25]:
param_dict = {
                'fig_width': 18,
                'fig_height': 12,
                'y_label_font': 45,
                'x_label_font': 45,
                 'x_ticks_font': 42,
                 'y_ticks_font': 50,    
                'legend_size': 30,
                'legend_title_size': 40,
                'title_size': 36,
                'title_pad': 40,             
             }

In [ ]:
naming_dict['title_prefix'] = "F1-Score (Macro Avg)"
plot_query_line_plot(stats_fscore, param_dict, naming_dict, color_dict)

In [ ]:
naming_dict['title_prefix'] = "False Alarm Rate"
plot_query_line_plot(stats_far, param_dict, naming_dict, color_dict)

In [ ]:
naming_dict['title_prefix'] = "Anomaly Miss Rate"
plot_query_line_plot(stats_amr, param_dict, naming_dict, color_dict)

## Old Slow Plot Codes

In [ ]:
for feature_count in unique_feature_counts:
    
    for num_train_known_apps in unique_known_train_apps:
        
        sub_df = result_df[(result_df['feature_count'] == feature_count) & (result_df['num_known_train_apps'] == num_train_known_apps)]
        sub_df = sub_df.sort_values(['method','query_strategy'])
        
        fig = plt.figure(figsize=(param_dict['fig_width'], param_dict['fig_height']))
        ax = sns.lineplot(x="query_iter", y="macro_avg_f1_score",
                          sort= False,
                          hue = 'query_strategy',
                          markers=True,
                          data=sub_df)

        legend = plt.legend(title="",prop={'size': param_dict['legend_size']})

        ax.set_title(f'Macro Avg F1-score w.r.t Number of Queries \n {FE_NAME} - {SYSTEM} - {int(feature_count)} features \n #Known Apps in Training: {num_train_known_apps}',fontsize=param_dict['title_size'])
        ax.set(ylim=(0.3, 1.01))

        ax.set_ylabel('F1-score',size=param_dict['y_label_font'])
        ax.set_xlabel("Number of Iterations",size=param_dict['x_label_font'])

        plt.xticks(fontsize=param_dict['x_ticks_font'])
        plt.yticks(fontsize=param_dict['y_ticks_font'])


        plt.tight_layout()  

In [41]:
for feature_count in unique_feature_counts:
    
    for num_train_known_apps in unique_known_train_apps:
        
        sub_df = result_df[(result_df['feature_count'] == feature_count) & (result_df['num_known_train_apps'] == num_train_known_apps)]
        sub_df = sub_df.sort_values(['method','query_strategy'])
        
        fig = plt.figure(figsize=(param_dict['fig_width'], param_dict['fig_height']))
        ax = sns.lineplot(x="query_iter", y="false_alarm_rate",
                          sort= False,
                          hue = 'query_strategy',
                          markers=True,
                          data=sub_df)

        legend = plt.legend(title="",prop={'size': param_dict['legend_size']})

        ax.set_title(f'False Alarm Rate w.r.t Number of Queries \n {FE_NAME} - {SYSTEM} - {int(feature_count)} features \n #Known Apps in Training: {num_train_known_apps}',fontsize=param_dict['title_size'])

        ax.set_ylabel('False Alarm Rate',size=param_dict['y_label_font'])
        ax.set_xlabel("Number of Iterations",size=param_dict['x_label_font'])

        plt.xticks(fontsize=param_dict['x_ticks_font'])
        plt.yticks(fontsize=param_dict['y_ticks_font'])


        plt.tight_layout()  

In [ ]:
for feature_count in unique_feature_counts:
    
    for num_train_known_apps in unique_known_train_apps:
        
        sub_df = result_df[(result_df['feature_count'] == feature_count) & (result_df['num_known_train_apps'] == num_train_known_apps)]
        sub_df = sub_df.sort_values(['method','query_strategy'])
        
        fig = plt.figure(figsize=(param_dict['fig_width'], param_dict['fig_height']))
        ax = sns.lineplot(x="query_iter", y="anomaly_miss_rate",
                          sort= False,
                          hue = 'query_strategy',
                          markers=True,
                          data=sub_df)

        legend = plt.legend(title="",prop={'size': param_dict['legend_size']})

        ax.set_title(f'Anomaly Miss Rate w.r.t Number of Queries \n {FE_NAME} - {SYSTEM} - {int(feature_count)} features \n #Known Apps in Training: {num_train_known_apps}',fontsize=param_dict['title_size'])

        ax.set_ylabel('Anomaly Miss Rate',size=param_dict['y_label_font'])
        ax.set_xlabel("Number of Iterations",size=param_dict['x_label_font'])

        plt.xticks(fontsize=param_dict['x_ticks_font'])
        plt.yticks(fontsize=param_dict['y_ticks_font'])

        plt.tight_layout()  

### Fixes

In [ ]:

read_dir =  conf['experiment_dir'] / 'CV_0' / 'unseen_app_results_one_repeat' / 'results'

for filename in os.listdir(read_dir):
    splitted_filename = filename.split('#')
        
    if splitted_filename[4] == 'random': 
        print(filename)    
        one_repeat_csv = pd.read_csv(read_dir / filename,index_col = [0])
        break
one_repeat_csv.head()        

In [ ]:
correct_cols = one_repeat_csv.columns

In [ ]:
read_dir =  conf['experiment_dir'] / 'CV_0' / 'unseen_app_results_one_repeat' / 'results'

for filename in os.listdir(read_dir):
    one_repeat_csv = pd.read_csv(read_dir / filename,index_col = [0])
    if not (one_repeat_csv.columns.values == correct_cols).all():
            print(filename)
            break

In [ ]:
# #Delete the wrong stuff
# for filename in os.listdir(conf['results_dir']):
#     splitted_filename = filename.split('#')
        
#     if splitted_filename[4] != 'random': 
#         print(filename)
#         #os.remove(conf['results_dir'] / filename)
#         #new_file = pd.read_csv(conf['results_dir'] / filename,index_col = [0])
#         #break
#new_file.head()

In [ ]:
read_dir =  conf['experiment_dir'] / 'CV_0' / 'unseen_app_results_worstbest' / 'results'

for filename in os.listdir(read_dir):
    prev_orig_csv = pd.read_csv(read_dir / filename,index_col = [0])
    break

In [ ]:
read_dir = conf['experiment_dir'] / 'CV_0' / 'unseen_app_results_worstbest' / 'results'

for filename in os.listdir(read_dir):
    
    splitted_filename = filename.split('#')
    
    if splitted_filename[4] != 'random':    
        lolo = pd.read_csv(read_dir / filename,index_col = [0])
        filename_splitted = filename.split('.')
        new_filename = filename_splitted[0] + '#0.' + filename_splitted[1]    
        num_repeat_col = lolo['num_repeat'].values
        query_iter_col = lolo['query_iter'].values
        lolo.drop(columns = ['num_repeat','query_iter'],inplace=True)
        lolo.insert(3, 'query_iter', query_iter_col)
        lolo.insert(4, 'repeat_num', num_repeat_col)        
        #print(new_filename)
        lolo.to_csv(conf['experiment_dir'] / 'CV_0' / 'unseen_app_results_worstbest' / 'modified_al_results' / new_filename)        
        #break

In [ ]:
read_dir =  conf['experiment_dir'] / 'CV_0' / 'unseen_app_results_worstbest' / 'modified_al_results'

for filename in os.listdir(read_dir):
    modified_csv = pd.read_csv(read_dir / filename,index_col = [0])
    
    if not (modified_csv.columns.values == correct_cols).all():
        print(filename)
        break